In [1]:
import pandas as pd

# Load the CSV file into a pandas DataFrame
file_path = 'gowalla_spots_subset1.csv'
spot_subset1 = pd.read_csv(file_path)

# Display the first few rows of the DataFrame to check if it's loaded correctly
print(spot_subset1.head(5))

     id            created_at         lng        lat  photos_count  \
0  8904  2008-12-06T16:28:53Z  -94.607499  39.052318             0   
1  8932  2008-12-13T02:16:51Z  -97.254356  32.927662             2   
2  8936  2008-12-14T22:08:39Z  -94.591995  39.053318             0   
3  8938  2008-12-15T00:22:49Z  -94.590311  39.052824            38   
4  8947  2008-12-16T23:14:05Z -122.029631  37.331880            91   

   checkins_count  users_count  radius_meters  highlights_count  items_count  \
0             114           21             35                 0           10   
1              67           48             75                 0            6   
2              75           46             75                 0           10   
3             438           94             50                10           10   
4            3100         1186            200                20           10   

   max_items_count                                    spot_categories  
0               10   [{'ur

In [2]:
# List of columns to drop
columns_to_drop = ["photos_count", "checkins_count", "users_count", 
                   "radius_meters", "highlights_count", "items_count", "max_items_count"]

# Drop the specified columns from the DataFrame
spot_subset1 = spot_subset1.drop(columns=columns_to_drop)

# Display the first few rows of the DataFrame to check the result
print(spot_subset1.head())

     id            created_at         lng        lat  \
0  8904  2008-12-06T16:28:53Z  -94.607499  39.052318   
1  8932  2008-12-13T02:16:51Z  -97.254356  32.927662   
2  8936  2008-12-14T22:08:39Z  -94.591995  39.053318   
3  8938  2008-12-15T00:22:49Z  -94.590311  39.052824   
4  8947  2008-12-16T23:14:05Z -122.029631  37.331880   

                                     spot_categories  
0   [{'url': '/categories/89', 'name': 'Craftsman'}]  
1         [{'url': '/categories/17', 'name': 'BBQ'}]  
2    [{'url': '/categories/103', 'name': 'Theatre'}]  
3  [{'url': '/categories/1', 'name': 'Coffee Shop'}]  
4  [{'url': '/categories/121', 'name': 'Corporate...  


In [3]:
# Get the number of unique user IDs in the "id" column
unique_user_ids = spot_subset1['id'].nunique()

# Display the result
print(f'There are {unique_user_ids} unique user IDs in the dataframe.')

There are 2724891 unique user IDs in the dataframe.


In [4]:
import pandas as pd
import json

# Load the JSON file
json_file_path = 'gowalla_category_structure.json'
with open(json_file_path, 'r') as f:
    categories_data = json.load(f)


In [5]:
# Initialize the category mapping
category_mapping = {}

# Recursive function to extract categories and subcategories
def extract_categories(category, main_category_name):
    # Extract main or sub-categories and add them to the mapping
    if 'url' in category:
        category_mapping[category['url']] = main_category_name
    
    # If there are sub-categories, recursively add them
    if 'spot_categories' in category:
        for sub_category in category['spot_categories']:
            extract_categories(sub_category, main_category_name)

# Iterate through the main categories and their sub-categories
for main_category in categories_data['spot_categories']:
    main_category_name = main_category['name']
    extract_categories(main_category, main_category_name)

# Check if all categories were mapped correctly
print(f"Total categories mapped: {len(category_mapping)}")

Total categories mapped: 266


In [6]:
import ast

# Function to extract the main category based on the sub-category URL, with error handling
# Function to extract the main category based on the sub-category URL or a direct string value
def get_main_category(category_info):
    try:
        # Check if the input is already a simple string (e.g., "Food", "Entertainment")
        if isinstance(category_info, str) and not category_info.startswith('['):
            return category_info  # Return the string directly if it's already the main category
        
        # If the input is a string representation of a list, attempt to parse it
        if isinstance(category_info, str):
            category_info = ast.literal_eval(category_info)  # Convert string to list/dict
        
        # Check if the parsed value is a list and contains at least one dictionary with a 'url' key
        if isinstance(category_info, list) and len(category_info) > 0 and 'url' in category_info[0]:
            sub_category_url = category_info[0]['url']
            return category_mapping.get(sub_category_url, 'Unknown')  # Map the sub-category to the main category
    except (ValueError, SyntaxError, KeyError) as e:
        # Handle any parsing errors or unexpected formats
        print(f"Error parsing category info: {category_info}, Error: {e}")
    
    return 'Unknown'

# Apply the function to the last column and create a new 'Category' column
spot_subset1['Category'] = spot_subset1.iloc[:, -1].apply(get_main_category)

# Display the updated DataFrame with the new 'Category' column
print(spot_subset1[['Category']].value_counts())

Category     
Food             680445
Shopping         656385
Community        496446
Outdoors         225057
Unknown          196938
Travel           195819
Entertainment    138333
Nightlife        135468
dtype: int64


In [7]:
print(spot_subset1.iloc[:, -1].head(10))

0        Community
1             Food
2    Entertainment
3             Food
4        Community
5          Unknown
6        Community
7         Outdoors
8          Unknown
9          Unknown
Name: Category, dtype: object


In [8]:
# Check the first few items of the category mapping
print(list(category_mapping.items())[:10])


[('/categories/934', 'Community'), ('/categories/133', 'Community'), ('/categories/135', 'Community'), ('/categories/138', 'Community'), ('/categories/153', 'Community'), ('/categories/139', 'Community'), ('/categories/140', 'Community'), ('/categories/141', 'Community'), ('/categories/142', 'Community'), ('/categories/143', 'Community')]


In [9]:
print(spot_subset1.iloc[:, -2].head(10))  # Assuming the last column with category info is second-to-last

0     [{'url': '/categories/89', 'name': 'Craftsman'}]
1           [{'url': '/categories/17', 'name': 'BBQ'}]
2      [{'url': '/categories/103', 'name': 'Theatre'}]
3    [{'url': '/categories/1', 'name': 'Coffee Shop'}]
4    [{'url': '/categories/121', 'name': 'Corporate...
5     [{'url': '/categories/452', 'name': 'Old Navy'}]
6    [{'url': '/categories/125', 'name': 'City Hall'}]
7     [{'url': '/categories/150', 'name': 'Fountain'}]
8     [{'url': '/categories/363', 'name': 'Chipotle'}]
9    [{'url': '/categories/903', 'name': 'Chick-fil...
Name: spot_categories, dtype: object


In [10]:
# Function to extract the main category based on the sub-category URL
def get_main_category(category_info):
    try:
        # If the input is a string representing a list, parse it
        if isinstance(category_info, str):
            category_info = ast.literal_eval(category_info)  # Convert string to list/dict
        
        # Check if it's a list and contains at least one dictionary with a 'url' key
        if isinstance(category_info, list) and len(category_info) > 0 and 'url' in category_info[0]:
            sub_category_url = category_info[0]['url']
            # Return the corresponding main category or 'Unknown' if not found
            return category_mapping.get(sub_category_url, 'Unknown')
    except (ValueError, SyntaxError, KeyError) as e:
        # Handle any parsing errors
        print(f"Error parsing category info: {category_info}, Error: {e}")
    
    return 'Unknown'

# Apply the function to the spot_categories column and create a new 'Category' column
spot_subset1['Category'] = spot_subset1['spot_categories'].apply(get_main_category)

# Display the updated DataFrame with the new 'Category' column
print(spot_subset1[['Category']].value_counts())

Category     
Food             680445
Shopping         656385
Community        496446
Outdoors         225057
Unknown          196938
Travel           195819
Entertainment    138333
Nightlife        135468
dtype: int64


In [11]:
print(spot_subset1.head(5))

     id            created_at         lng        lat  \
0  8904  2008-12-06T16:28:53Z  -94.607499  39.052318   
1  8932  2008-12-13T02:16:51Z  -97.254356  32.927662   
2  8936  2008-12-14T22:08:39Z  -94.591995  39.053318   
3  8938  2008-12-15T00:22:49Z  -94.590311  39.052824   
4  8947  2008-12-16T23:14:05Z -122.029631  37.331880   

                                     spot_categories       Category  
0   [{'url': '/categories/89', 'name': 'Craftsman'}]      Community  
1         [{'url': '/categories/17', 'name': 'BBQ'}]           Food  
2    [{'url': '/categories/103', 'name': 'Theatre'}]  Entertainment  
3  [{'url': '/categories/1', 'name': 'Coffee Shop'}]           Food  
4  [{'url': '/categories/121', 'name': 'Corporate...      Community  


In [12]:
# List of columns to drop
columns_to_drop = ["spot_categories"]

# Drop the specified columns from the DataFrame
spot_subset1 = spot_subset1.drop(columns=columns_to_drop)

# Display the first few rows of the DataFrame to check the result
print(spot_subset1.head())

     id            created_at         lng        lat       Category
0  8904  2008-12-06T16:28:53Z  -94.607499  39.052318      Community
1  8932  2008-12-13T02:16:51Z  -97.254356  32.927662           Food
2  8936  2008-12-14T22:08:39Z  -94.591995  39.053318  Entertainment
3  8938  2008-12-15T00:22:49Z  -94.590311  39.052824           Food
4  8947  2008-12-16T23:14:05Z -122.029631  37.331880      Community


In [13]:
# Step 1: Load the space-delimited text file into a DataFrame
# Replace 'path_to_checkins.txt' with the actual path to your checkins text file
checkins_data = pd.read_csv('Gowalla_totalCheckins.txt', delim_whitespace=True, header=None, names=['user_id', 'time', 'lat', 'lng', 'pid'])


In [14]:
# Step 2: Rename the 'created_at' column to 'time' and 'userid' to 'user_id' in spot_subset1
spot_subset1 = spot_subset1.rename(columns={'created_at': 'time', 'id': 'user_id'})

In [15]:
checkins_data['time'] = pd.to_datetime(checkins_data['time'], errors='coerce', utc=True)
spot_subset1['time'] = pd.to_datetime(spot_subset1['time'], errors='coerce', utc=True)

In [16]:
print(spot_subset1.head())

   user_id                      time         lng        lat       Category
0     8904 2008-12-06 16:28:53+00:00  -94.607499  39.052318      Community
1     8932 2008-12-13 02:16:51+00:00  -97.254356  32.927662           Food
2     8936 2008-12-14 22:08:39+00:00  -94.591995  39.053318  Entertainment
3     8938 2008-12-15 00:22:49+00:00  -94.590311  39.052824           Food
4     8947 2008-12-16 23:14:05+00:00 -122.029631  37.331880      Community


In [17]:
spot_subset1 = spot_subset1.rename(columns={'pid': 'user_id', 'id': 'user_id'})

In [37]:
print(checkins_data.head())

   user_id                      time        lat        lng      pid
0        0 2010-10-19 23:55:27+00:00  30.235909 -97.795140    22847
1        0 2010-10-18 22:17:43+00:00  30.269103 -97.749395   420315
2        0 2010-10-17 23:42:03+00:00  30.255731 -97.763386   316637
3        0 2010-10-17 19:26:05+00:00  30.263418 -97.757597    16516
4        0 2010-10-16 18:50:42+00:00  30.274292 -97.740523  5535878


In [19]:
# Load the CSV file into a pandas DataFrame
file_path = 'gowalla_checkins.csv'
original_checkin = pd.read_csv(file_path)

# Display the first few rows of the DataFrame to check if it's loaded correctly
print(original_checkin.head(5))

   userid  placeid              datetime
0    1338   482954  2011-06-23T02:24:22Z
1    1338   580963  2011-06-22T14:23:03Z
2    1338   365256  2011-06-09T23:29:30Z
3    1338    89504  2011-05-22T15:54:30Z
4    1338  1267135  2011-05-21T16:51:13Z


In [20]:
original_checkin = original_checkin.rename(columns={'userid': 'user_id', 'placeid': 'pid', 'datetime': 'time'})

In [38]:
spot_subset1 = spot_subset1.rename(columns={'user_id': 'pid'})

In [26]:
original_checkin['time'] = pd.to_datetime(checkins_data['time'], errors='coerce', utc=True)

In [40]:
# Step 1: Load the checkin data (if not already loaded)
# Assuming 'checkin_data' contains 'user_id', 'pid', 'time' columns

# Step 2: Perform a left join using 'pid' as the key
# We are joining on 'pid' to add the 'lat', 'lng', and 'category' columns to checkin_data
merged_df = pd.merge(original_checkin, spot_subset1[['pid', 'lat', 'lng', 'Category']], 
                     on='pid', how='left')

# Step 3: Display the resulting dataframe
print(merged_df.head())

# Optionally, check the number of rows in the resulting dataframe
print(f"Number of rows in merged dataframe: {len(merged_df)}")

   user_id      pid                      time        lat        lng  \
0     1338   482954 2010-10-19 23:55:27+00:00  44.934198 -93.177506   
1     1338   580963 2010-10-18 22:17:43+00:00  45.204928 -93.366249   
2     1338   365256 2010-10-17 23:42:03+00:00  44.981997 -93.277574   
3     1338    89504 2010-10-17 19:26:05+00:00  44.982573 -93.153680   
4     1338  1267135 2010-10-16 18:50:42+00:00  44.948173 -93.187120   

        Category  
0      Nightlife  
1       Shopping  
2  Entertainment  
3  Entertainment  
4       Shopping  
Number of rows in merged dataframe: 36001959


In [43]:
# Count the number of rows where pid is NaN
num_nan = merged_df['Category'].isna().sum()

# Display the result
print(f"Number of rows  as NaN: {num_nan}")

Number of rows  as NaN: 325678


In [44]:
print(merged_df[['Category']].value_counts())

Category     
Food             7853482
Shopping         7003264
Community        6422122
Travel           3959631
Unknown          3434614
Outdoors         2673774
Entertainment    2326856
Nightlife        2002538
dtype: int64


In [45]:
# Step 1: Drop all rows with any NaN or missing values
cleaned_df = merged_df.dropna()

# Step 2: Display the cleaned dataframe
print(cleaned_df.head())

# Step 3: Optionally, check the number of rows in the cleaned dataframe
print(f"Number of rows after dropping NaN values: {len(cleaned_df)}")

   user_id      pid                      time        lat        lng  \
0     1338   482954 2010-10-19 23:55:27+00:00  44.934198 -93.177506   
1     1338   580963 2010-10-18 22:17:43+00:00  45.204928 -93.366249   
2     1338   365256 2010-10-17 23:42:03+00:00  44.981997 -93.277574   
3     1338    89504 2010-10-17 19:26:05+00:00  44.982573 -93.153680   
4     1338  1267135 2010-10-16 18:50:42+00:00  44.948173 -93.187120   

        Category  
0      Nightlife  
1       Shopping  
2  Entertainment  
3  Entertainment  
4       Shopping  
Number of rows after dropping NaN values: 6396728


In [46]:
print(cleaned_df[['Category']].value_counts())

Category     
Food             1426229
Shopping         1291932
Community        1092153
Travel            664029
Unknown           640035
Outdoors          498561
Entertainment     425714
Nightlife         358075
dtype: int64


In [47]:
# Step 1: Group the data by 'Category'
grouped = cleaned_df.groupby('Category')

# Step 2: Save each group as a separate CSV file
for category, group_data in grouped:
    # Generate the file name based on the category
    file_name = f"{category.lower().replace(' ', '_')}_data.csv"
    
    # Save the group data to a CSV file
    group_data.to_csv(file_name, index=False)
    
    print(f"Saved {file_name}")

Saved community_data.csv
Saved entertainment_data.csv
Saved food_data.csv
Saved nightlife_data.csv
Saved outdoors_data.csv
Saved shopping_data.csv
Saved travel_data.csv
Saved unknown_data.csv
